# Tennis Match Simulation Problem

>[Tennis Match Simulation Problem](#updateTitle=true&folderId=1I6abHxBfYUvjHkeR_NTOIAxOrl74iEy2&scrollTo=1OuxDx4DJuZF)

>>>[Theoretical Probability of winning a single game](#updateTitle=true&folderId=1I6abHxBfYUvjHkeR_NTOIAxOrl74iEy2&scrollTo=RHP2gBkgIbeW)

>>>[Let's simulate and check if the result is correct](#updateTitle=true&folderId=1I6abHxBfYUvjHkeR_NTOIAxOrl74iEy2&scrollTo=mA-gKU3SIeij)

>>[Simulate who wins a tie-breaker](#updateTitle=true&folderId=1I6abHxBfYUvjHkeR_NTOIAxOrl74iEy2&scrollTo=OGM6KgDzIlRN)

>>[Simulate who wins a single set](#updateTitle=true&folderId=1I6abHxBfYUvjHkeR_NTOIAxOrl74iEy2&scrollTo=DR5AruJ3I2wF)

>>[Simulate the result of a match](#updateTitle=true&folderId=1I6abHxBfYUvjHkeR_NTOIAxOrl74iEy2&scrollTo=Pohk0_zwJGLb)

>>>[Simulate the probability of player winning a match and its confidence interval](#updateTitle=true&folderId=1I6abHxBfYUvjHkeR_NTOIAxOrl74iEy2&scrollTo=xCt_8tSsJao9)



We are interested in predicting tennis matches. While there are several approaches to do this, we want to take a point-by-point approach. Assume that a player's serve is iid. Let $p_{s1}$ be the probability that a player 1 will win a point on his serve and $p_{s2}$ the probability that player 2 will win a point on his serve. If one player wins a point, the other player has to lose a point so the probability of winning a return for player 1 is just $p_{r1} = 1 - p_{s2}$. We can therefore forget about return probabilities. While you can work out the probability of winning a match with recursive equation and a bit of combinatorics, we want to simulate this instead. We will that all sets at 6 games each end in tiebreaks and we play best of 3 sets. If you are less familiar with tennis, you can find the rules here : https://en.wikipedia.org/wiki/Tennis_scoring_system

1. Write an algorithm to simulate the probability of winning the match for probabilities $p_{s1}$ and $p_{s2}$. You should have a function addPoint(player) that adds one point for a player. You could use an integer for player and just let `'1'` designate player 1 and `'2'` player 2. You should have a function addPoints(points) that takes an array/vector of points and adds them in sequence. You can follow the above format. Finally you should have a function getScore() that prints the score. It should be in the format 40-15 | 6:3 6:7 3:3, i.e. for games in progress and tiebreaks, list the player serving first. For sets ( whether in progress or completed), list player 1 first and then player 2. 

2. What are the probabilities of winning the match with $p_{s1} = 0.64$ and $p_{s2} = 0.62$? How many runs do you need? What is your confidence interval?

3. Let's expand the simulation. Assume the probability of winning the point changes on `'big points'`. A `'big point'` is defined as a point that can win you a game or set (so includes set points in tie breaks). So we add the probabilities $p_{s1, B}$ and $p_{s2, B}$ as the probability that a player will win his point on serve on big point. Otherwise the probabilities remain $p_{s1}$ and $p_{s2}$. 


In [0]:
import random
import numpy as np

def play_a_single_game(ps, psB = None):
  """
  Function to check if a given player with 'ps' chances of winning a 
  given point wins a single game. 
  
  Input : 
  ----------------
  ps - probability of server winning a single point 
  psB - probability of winning a big point with default None
  
  Output : 
  ----------------
  a list of list like [1, [40,30]] where
  
  first element --- whether she wins a single game or not
  second element --- score of the current game 
  """
  # set their respective points to zero; here we will 
  # represent 0 by 0, 15 by 1, 30 by 2 and 40 by 3 and 40+ by 4. 
  # I store this dictionary for finally returning the score of the game
  # note that I denote 40+ score by 4 as well 
  score_dict = {0:0, 1:15, 2:30, 3:40, 4:40} 
  score1 = 0 
  score2 = 0
  # set the big point probabilities to original if not provided
  psB = ps if psB is None else psB
  
  while True:
    # change the original probability to big point probability 
    # if it is the game point i.e 40 for the server
    ps = psB if score1 == 3 else ps 
    # simulate the bernoulli trial for the given player 
    # and check if she wins a point or not
    if random.random() < ps:
      # if she wins a point increment her score by 1
      score1 += 1
    else:
      # otherwise increment the score of her opponent
      score2 += 1
    # print score1,'-', score2
    # winning condition for the given player 
    if score1 >= 4 and (score1-score2) >= 2: 
      # if the score of first player is 4 and the difference 
      # between the score of two players 
      return [1, [score_dict[score1], score_dict[score2]]]
    # winning condition for the opponent of the given player
    elif score2 >= 4 and (score2-score1) >= 2:
      return [0, [score_dict[score1], score_dict[score2]]]
   # if there is deuce or advantage score e.g. 6-5,
  # we bring it to a score like 3-2 since this is equivalent
    while True:
      if (score1 + score2) > 6:
        # we set the original probability to big point prob
          ps = psB
          score1 -= 1
          score2 -= 1
      else:   
        break

In [0]:
random.seed(100)
play_a_single_game(0.64)

### Theoretical Probability of winning a single game

In [0]:
# I have created below function to check my above code 

def actual_probability_of_winning_game(ps):
  """
  Function computes the actual probability of a server winning a game 
  if the probability of winning a single point is known.
  
  Input : ps - probability of server winning a single point 
  
  Output : the probability of winning a game   
  
  citation 
  ---------------------------
  https://pdfs.semanticscholar.org/e870/8ca24b8f67476b3284ec22ccb689dc6229be.pdf
  
  Probability of Winning at Tennis I. Theory and Data
  By Paul K. Newton and Joseph B. Keller
  """
  result = ps**4 + 4*(ps**4)*(1-ps) + 10*(ps**4)*(1-ps)**2 + 20*(ps**5)*((1-ps)**3)/(1-2*ps*(1-ps))
  return result

### Let's simulate and check if the result is correct

In [0]:
# Lets do some simulations and check if the simulated result is close to the 
# theoretical probability
n = 10000000
x = range(n)
for i in range(n):
  x[i] = play_a_single_game(0.9)[0]

# calculate the empirical probability of winning a single game 
p_hat = sum(x)/float(n)
# calculate the corresponding standard deviation
sd = np.sqrt(p_hat*(1-p_hat)/float(n))

print 'The simulated probability is ', p_hat ,' with standard error ', sd, '\n'

print "The 95 percent confidence interval for the true value of p is (%5f, %5f)" %(p_hat - 1.96*sd, p_hat + 1.96*sd),'\n'

print 'The actual probability is ', actual_probability_of_winning_game(0.9)

> We can observe from the above empirical finding that the 95% confidence interval contains the true value of probability of winning a game. This empirical evidence suggests that the above simulation of a game is correct.

## Simulate who wins a tie-breaker

In [0]:
def play_a_tiebreaker(ps1, ps2, server = 1):
  """
  Function simulates the result of a tie-breaker game and returns if
  player 1 has won the tie-breaker or not. Here I am assuming that a
  tie-breaker consists of 7-points and a player wins a tie-breaker if 
  she wins by a difference of 2 points. Here points are counted as 0,
  1,2,3,4,5,6,7.
  
  Input : 
  --------------------
  ps1 - probability of player 1 winning a single point if she serves
  ps2 - probability of player 2 winning a single point if she serves 
  server - a bool representing who is serving at the start of the tie breaker.
  
  Output :
  --------------------
  Returns 1/0 representing if player 1 wins a given tie breaker. 
  
  """
  # set 
  next_set_server = 2 if server == 1 else 1
  # set their respective points to zero;  
  tiebreaker_score1 = 0 
  tiebreaker_score2 = 0
  while True:
    if server == 1: 
      # check if player 1 is serving and 
      # next check if she won the game 
      if random.random() < ps1:
        # increment her score by 1 
        tiebreaker_score1 += 1
        # and change the server to player 2
        server = 2
      else:
        # if player 1 did not win the game then increment player 2 score
        tiebreaker_score2 += 1
        # since player 1 was serving, we allot serve to player 2
        server = 2
    else:
      # if player 2 is serving then check if she won the game and store the 
      if random.random() < ps2:
        # if she won the game then increment her score by 1
        tiebreaker_score2 += 1
        # change the server to player 1
        server = 1
      else:
        # if player 2 did not win the game then increment the score of player 2
        tiebreaker_score1 += 1
        # allot serve to player 1
        server = 1
    # print tiebreaker_score1, '-', tiebreaker_score2
    if tiebreaker_score1 >= 7 and (tiebreaker_score1-tiebreaker_score2) >= 2:
        return 1
    elif tiebreaker_score2 >= 7 and (tiebreaker_score2-tiebreaker_score1) >= 2:
        return 0
    # if the tie breaker score become something like 8:7 then we bring it back
    # to 6:5 for simplicity. These two scores are equivalent.
    while True:
        if (tiebreaker_score1+tiebreaker_score2) > 12:
            tiebreaker_score1 -= 1
            tiebreaker_score2 -= 1
        else:   
          break

In [0]:
# play a tie-breaker game 
play_a_tiebreaker(0.64, 0.62, server=1)

## Simulate who wins a single set

In [0]:
def play_a_single_set(ps1, ps2, ps1B = None, ps2B = None, server = 1):
  """
  Function checks if a server wins a given set. 
  
  Input : 
  --------------------
  ps1 - probability of player 1 winning a single point if she serves
  ps2 - probability of player 2 winning a single point if she serves 
  ps1B - probability of player 1 winning a big point with default None
  ps2B - probability of player 2 winning a big point with default None
  server - a bool representing who is serving at the start of the set.
  
  Output : a list of lists consisting like [0, [40, 30], [6, 7], 1]
  --------------------
  first element  : outcome of the set if first player wins or not
  second element : score of the last game played 
  third element  : score of the current set 
  fourth element : who will be serving next 1 for player 1 and 2 for player 2.
  --------------------
  Therefore [0, [40, 30], [6, 7], 1] means that first player lost current set 
  to player 2 with a score of 6-7 while in the last game they played, the score
  was 40:30 which was won by player 2, therefore next turn to serve is of player 1.
  """
  # set big point probabilities if not provided 
  ps1B = ps1 if ps1B is None else ps1B
  ps2B = ps2 if ps2B is None else ps2B
  
  # set the initial set score of each player to zero
  set_score1 = 0
  set_score2 = 0
  # repeat until a result is obtained 
  while True:
    if server == 1: 
      # check if player 1 is serving and save the outcome of her game
      check_if_won_game = play_a_single_game(ps1, ps1B)
      # print check_if_won_game[1]
      # next check if she won the game 
      if check_if_won_game[0] == 1:
        # increment her score by 1 
        set_score1 += 1
        # and change the server to player 2
        server = 2
      else:
        # if player 1 did not win the game then increment player 2 score
        set_score2 += 1
        # since player 1 was serving, we allot serve to player 2
        server = 2
    else:
      # if player 2 is serving then check if she won the game and store the 
      # result of the game if she is serving
      check_if_won_game = play_a_single_game(ps2, ps2B)
      # print check_if_won_game[1]
      # check if she won the game 
      if check_if_won_game[0] == 1:
        # if she won the game then increment her score by 1
        set_score2 += 1
        # change the server to player 1
        server = 1
      else:
        # if player 2 did not win the game then increment the score of player 2
        set_score1 += 1
        # allot serve to player 1
        server = 1
    # print set_score1,'-', set_score2
    # check if a tie break has occurred 
    if set_score1 == 6 and set_score2 == 6:
      # if it is a tie break then play a tie break with big point probabilities
      result_tiebreaker = play_a_tiebreaker(ps1B, ps2B, server)
      # change the server after the tie breaker is over 
      server = 2 if server == 1 else 1
      # if player 1 has won the tie breaker then increment her score by 1 and
      # return 1
      if result_tiebreaker == 1:
        set_score1 += 1
        # return player 1 won the set together with score of last game played
        # set scores of two players and whose turn is it to serve next
        return [1, check_if_won_game[1],[set_score1, set_score2], server]
      # if player 1 did not win then increment score of player 2 and return 0
      # together with set scores, and the next server
      else:
        set_score2 += 1
        return [0,check_if_won_game[1], [set_score1, set_score2], server]
    # check if the game is already over and player 1 has won it then 
    # return 1 together with scores mentioned above.
    if set_score1 >= 6 and (set_score1-set_score2) >= 2:
      return [1, check_if_won_game[1], [set_score1, set_score2], server]
    # if player 2 won it then do similarly 
    elif set_score2 >= 6 and (set_score2-set_score1) >= 2:
      return [0,check_if_won_game[1], [set_score1, set_score2], server]
    # if score becomes something like 8:7 then bring it back to 6:5 since the 
    # two scores are equivalent
    while True:
      if (set_score1 + set_score2) > 12:
        # since this must have been a tie-breaker situation therefore we turn on
        # the big probabilities namely 
        ps1 =  ps1B 
        ps2 =  ps2B
        # decrement the scores of 8:7 to 6:5 which is equivalent
        set_score1 -= 1
        set_score2 -= 1
      else:
        break         

> In the result below we simulate a set where player 1 lost the set 4-6 to player 2 while it is player 2 turn to serve next. The result of their last game where player 1 was serving was 15:40 where player 1 lost to player 2.

In [0]:
random.seed(100)
play_a_single_set(0.64,0.62,2)

## Simulate the result of a match

In [0]:
def play_a_match(ps1, ps2, ps1B = None, ps2B = None, server = 1, no_of_sets = 3):
  """
  Function simulates a match and returns if player 1 won the match or not
  
  Input : 
  --------------------
  ps1 - probability of player 1 winning a single point if she serves
  ps2 - probability of player 2 winning a single point if she serves 
  ps1B - probability of player 1 winning a big point with default None
  ps2B - probability of player 2 winning a big point with default None
  server - a bool representing who is serving at the start of the match.
  no_of_sets - total number of sets in the match default value 3.
  
  Output :
  --------------------
  Returns a list of list consisting of the following :
  first element  : 1/0 representing if player 1 wins this match or not. 
  second element : the result of last game where current server is listed first
  next element   : the scores of the sets played between them [score1, score2]
  """
  # set the big probabilities to original prob if not supplied
  ps1B = ps1 if ps1B is None else ps1B
  ps2B = ps2 if ps2B is None else ps2B
  # set the number of sets won by each player to 0
  no_of_sets_won1 = 0
  no_of_sets_won2 = 0
  # container to store the result of each set.
  sets = []
  # container to store the result of last game 
  last_game_result = []
  # loop over all number of sets until a player has won the match
  for i in range(no_of_sets):
    # play individual sets and store the result 
    set_result = play_a_single_set(ps1, ps2, ps1B, ps2B, server)
    # store the result of last set played 
    sets.append(set_result[2])
    # store the result of the last game played 
    last_game_result = set_result[1]
    # set who will be serving next 
    server = set_result[3]
    # increase the counter of number of sets won by each player 
    no_of_sets_won1 += set_result[0]
    # if player 1 wins a set then player 2 win 1-1 = 0 set 
    no_of_sets_won2 += 1-set_result[0]
    # exit if one of them reaches to 2 sets win
    if(no_of_sets_won1==2 or no_of_sets_won2==2):
      break
  match_result = (no_of_sets_won1 > no_of_sets_won2)*1
  return [match_result, last_game_result, sets]

>The output below is a simulated match between player 1 and player 2 where player 1 won that match with scores of 6:2, 2:6 and 6:3 while the result of the last game between then was 40-15.

In [0]:
random.seed(100)
play_a_match(0.64,0.62, 0.64,0.62)

In [0]:
def get_score_of_match(ps1, ps2, ps1B = None, ps2B = None, server = 1, no_of_sets = 3):
  """
  A wrapper function for play_a_match() which outputs the result of a tennis match in a nice fashion 
  
  Input : similar to play_a_match (refer above for more details )
  
  Output : prints the result on screen for player 1
  """
  # get the match result and store it 
  m = play_a_match(ps1,ps2,ps1B, ps2B, server, no_of_sets)
  # extract the individual set scores 
  s = m[2]
  if len(s)== no_of_sets -1:
    print "%d-%d | %d:%d %d:%d " %(m[1][0], m[1][1],s[0][0], s[0][1], s[1][0], s[1][1] )
  else:
    print "%d-%d | %d:%d %d:%d %d:%d" %(m[1][0], m[1][1],s[0][0], s[0][1], s[1][0], s[1][1],s[2][0], s[2][1] )

In [0]:
# print the score in a nice fashion
get_score_of_match(0.64,0.62)

### Simulate the probability of player winning a match and its confidence interval

In [0]:
# We set the number of simulation to a large value for the central limit 
# theorem to hold
n = 1000000
x = [play_a_match(0.64, 0.62, 0.64,0.62)[0] for i in range(n)]

# get the estimated value of the probability of winning a match 
p_hat = sum(x)/float(n)

# calculate the standard error of p_hat
sd = np.sqrt(p_hat*(1-p_hat)/float(n))

print 'The simulated probability is ', p_hat ,' with standard error ', sd, '\n'

print 'The 95 percent confidence interval of the actual probability is (%5f, %5f)'%(p_hat - 1.96*sd, p_hat + 1.96*sd)